In [1]:
import warnings
warnings.filterwarnings('ignore')


In [2]:
import pandas as pd
from bs4 import BeautifulSoup
import nltk
nltk.download('punkt')  # Download for sentence tokenization
nltk.download('stopwords')  # Download for stopword removal


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ecesoclab\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ecesoclab\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
df = pd.read_csv('Reviews.csv')


In [4]:
df.dropna(subset=['Summary'], inplace=True)

In [5]:
import pandas as pd
from bs4 import BeautifulSoup
import nltk
import string

# Initialize NLTK resources before your functions
nltk.download('punkt')
nltk.download('stopwords')

# Get the list of English stopwords
stop_words = set(nltk.corpus.stopwords.words('english'))
punctuation_table = str.maketrans('', '', string.punctuation)

def clean_html(text):
    soup = BeautifulSoup(text, 'html.parser')
    return soup.get_text()

def tokenize_and_remove_stopwords(text):
    tokens = nltk.word_tokenize(text)
    filtered_tokens = [w for w in tokens if w.lower() not in stop_words]
    return filtered_tokens

# Define the enhanced preprocessing function
def preprocess_text(text):
    text = text.lower()
    tokens = tokenize_and_remove_stopwords(text)
    tokens = [word for word in tokens if word not in stop_words]
    tokens = [word.translate(punctuation_table) for word in tokens]
    tokens = [word for word in tokens if word.strip()]
    return ' '.join(tokens)

# Apply preprocessing to your DataFrame
df['Processed_Summary'] = df['Summary'].apply(clean_html)
df['Processed_Summary'] = df['Processed_Summary'].apply(preprocess_text)  # Enhanced preprocessing

df['Processed_Text'] = df['Text'].apply(clean_html)
df['Processed_Text'] = df['Processed_Text'].apply(preprocess_text)  # Enhanced preprocessing


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ecesoclab\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ecesoclab\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
df = df.reset_index(drop=True)


In [7]:
from sklearn.model_selection import train_test_split

# Split the data
train_df, test_df = train_test_split(df, test_size=0.25, random_state=42)


In [8]:
train_df = train_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)

In [9]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel

tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token
model = GPT2LMHeadModel.from_pretrained('gpt2')

# Now you can use the tokenizer with padding
inputs = tokenizer("Example text", return_tensors="pt", padding=True)

In [10]:
from torch.utils.data import Dataset

class SummaryDataset(Dataset):
    def __init__(self, texts, summaries, tokenizer, max_length=256):
        self.texts = texts
        self.summaries = summaries
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        summary = self.summaries[idx]
        encoding = tokenizer.encode_plus(
            text, summary,
            add_special_tokens=True,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        return {
            'input_ids': encoding.input_ids.flatten(),
            'attention_mask': encoding.attention_mask.flatten(),
            'labels': encoding.input_ids.flatten()
        }



In [11]:
from transformers import TrainingArguments, Trainer

# Adjusted training arguments for CPU
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=1,  # Reduced for faster testing cycles
    per_device_train_batch_size=2,  # Reduced to ensure it fits into CPU memory
    per_device_eval_batch_size=2,
    warmup_steps=100,  # Reduced warmup steps
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,  
    evaluation_strategy="epoch",  # Evaluate at the end of each epoch to save time during training
    save_strategy="no", 
    load_best_model_at_end=False,
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=SummaryDataset(train_df['Processed_Text'], train_df['Processed_Summary'], tokenizer),
    eval_dataset=SummaryDataset(test_df['Processed_Text'], test_df['Processed_Summary'], tokenizer)
)

# Start training
trainer.train()


  1%|          | 10/1125 [00:18<33:38,  1.81s/it]

{'loss': 6.8677, 'grad_norm': 196.80755615234375, 'learning_rate': 5e-06, 'epoch': 0.01}


  2%|▏         | 20/1125 [00:38<38:15,  2.08s/it]

{'loss': 4.4982, 'grad_norm': 136.36334228515625, 'learning_rate': 1e-05, 'epoch': 0.02}


  3%|▎         | 30/1125 [00:59<37:53,  2.08s/it]

...

100%|█████████▉| 1120/1125 [38:06<00:10,  2.03s/it]

{'loss': 1.3586, 'grad_norm': 1.4782109260559082, 'learning_rate': 2.439024390243903e-07, 'epoch': 1.0}


100%|██████████| 1125/1125 [38:16<00:00,  2.02s/it]Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
                                             

{'eval_loss': 1.1394414901733398, 'eval_runtime': 210.7468, 'eval_samples_per_second': 3.559, 'eval_steps_per_second': 1.779, 'epoch': 1.0}
{'train_runtime': 2506.9478, 'train_samples_per_second': 0.898, 'train_steps_per_second': 0.449, 'train_loss': 1.3194318618774414, 'epoch': 1.0}


TrainOutput(global_step=1125, training_loss=1.3194318618774414, metrics={'train_runtime': 2506.9478, 'train_samples_per_second': 0.898, 'train_steps_per_second': 0.449, 'train_loss': 1.3194318618774414, 'epoch': 1.0})

In [ ]:
!pip install rouge-score



In [14]:
from rouge_score import rouge_scorer



# Function to generate summaries
def generate_summary(model, tokenizer, text, max_length=256):
    try:
        input_ids = tokenizer.encode(text, return_tensors='pt')
        if input_ids.size(1) > max_length:
            print(f"Skipping generation for input of length {input_ids.size(1)} as it exceeds max_length of {max_length}.")
            return None

        summary_ids = model.generate(input_ids, max_length=max_length, num_beams=5, early_stopping=True)
        return tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    
    except Exception as e:
        print(f"An error occurred: {str(e)}")
        return None
def compute_rouge_scores(model, dataset, tokenizer, dataframe):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    scores_list = []
    for i in range(len(dataset)):
        text = dataset.texts[i]
        reference_summary = dataset.summaries[i]
        generated_summary = generate_summary(model, tokenizer, text)
        if generated_summary is None:
            scores_list.append({key: None for key in ['ROUGE-1 Precision', 'ROUGE-1 Recall', 'ROUGE-1 F1', 'ROUGE-2 Precision', 'ROUGE-2 Recall', 'ROUGE-2 F1', 'ROUGE-L Precision', 'ROUGE-L Recall', 'ROUGE-L F1']})
            continue
        scores = scorer.score(reference_summary, generated_summary)
        scores_list.append({
            'ROUGE-1 Precision': scores['rouge1'].precision,            
            'ROUGE-1 Recall': scores['rouge1'].recall,
            'ROUGE-1 F1': scores['rouge1'].fmeasure,
            'ROUGE-2 Precision': scores['rouge2'].precision,
            'ROUGE-2 Recall': scores['rouge2'].recall,
            'ROUGE-2 F1': scores['rouge2'].fmeasure,
            'ROUGE-L Precision': scores['rougeL'].precision,
            'ROUGE-L Recall': scores['rougeL'].recall,
            'ROUGE-L F1': scores['rougeL'].fmeasure
        })

    # Append scores to the dataframe
    score_df = pd.DataFrame(scores_list)
    enhanced_df = pd.concat([dataframe.reset_index(drop=True), score_df], axis=1)
    return enhanced_df



# Initialize the datasets
train_dataset = SummaryDataset(train_df['Processed_Text'], train_df['Processed_Summary'], tokenizer)
test_dataset = SummaryDataset(test_df['Processed_Text'], test_df['Processed_Summary'], tokenizer)

# Evaluate ROUGE scores on the test set
final_scores_df = compute_rouge_scores(model, test_dataset, tokenizer, test_df)

# Save the final DataFrame with ROUGE scores to CSV
final_scores_df.to_csv("final_rouge_scores_2nd_Setting.csv", index=False)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generati

Skipping generation for input of length 329 as it exceeds max_length of 256.


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Skipping generation for input of length 359 as it exceeds max_length of 256.


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generati

Skipping generation for input of length 276 as it exceeds max_length of 256.


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generati

Skipping generation for input of length 531 as it exceeds max_length of 256.


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generati

In [15]:
final_scores_df

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,...,Processed_Text,ROUGE-1 Precision,ROUGE-1 Recall,ROUGE-1 F1,ROUGE-2 Precision,ROUGE-2 Recall,ROUGE-2 F1,ROUGE-L Precision,ROUGE-L Recall,ROUGE-L F1
0,476269,B000SQLQ0Y,A31RSJTGLVV3TR,T. Wayne,5,8,1,1304812800,Made in China - With CANCER,If you don't mind the inevitable increased can...,...,nt mind inevitable increased cancer risk andor...,0.062500,1.000000,0.117647,0.021277,0.5,0.040816,0.041667,0.666667,0.078431
1,288473,B000ENUC3S,A2QN7FECIWB7D2,"Pm Rodgers ""pmiker""",0,1,5,1312070400,Real Cherry Flavor,While I did not at any time imagine that I was...,...,time imagine eating real piece pie flavor real...,0.078947,1.000000,0.146341,0.027027,0.5,0.051282,0.052632,0.666667,0.097561
2,199206,B0002NV04K,A26EXMDN188M0,Lysan,2,2,4,1299715200,A good buy...,"...money wise, in any case. They're much cheap...",...,money wise case re much cheaper stores area dr...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
3,229917,B000ZSX4GE,AM9LRYG6YXV83,"P. Verkhovensky ""uebermensch""",5,5,5,1212451200,"Unique product, well executed",The flavor of these nuts is similar to cinnamo...,...,flavor nuts similar cinnamon toast cereal yet ...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
4,271145,B000GG0BQ6,ABDDWK3DS2S5C,Amy Y. Chung,0,0,5,1308873600,Great Green Tea by Bigelow,This is a wonderful product by Bigelow. It he...,...,wonderful product bigelow helps keep bones str...,0.117647,0.500000,0.190476,0.000000,0.0,0.000000,0.058824,0.250000,0.095238
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
745,558549,B000A11IRI,AL6V1X50WLAMY,Michelle Rama,8,8,5,1194048000,Awesome!,All of Ferdies products are delicious and well...,...,ferdies products delicious well worth price or...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
746,337262,B007OSBE1U,A2OEBM6XH6K7P4,The Glasscocks,2,3,5,1348531200,Great coffee experience,"Great coffee, very bold and smooth. It was ex...",...,great coffee bold smooth excellent price best ...,0.133333,0.666667,0.222222,0.071429,0.5,0.125000,0.133333,0.666667,0.222222
747,192723,B0031UBWLM,AOOIQU9NS7XU5,Jenny Miller,2,2,5,1330387200,Like Crack for Babies,My 6 month old son's favorite flavor. Wheneve...,...,6 month old son s favorite flavor whenever tee...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
748,426034,B000TQEWM2,APHXWZJ22M17M,T. Stratton,0,0,4,1210204800,Keurig Coffee,I like this coffee a lot. Also the Gloria Jea...,...,like coffee lot also gloria jean variety pack ...,0.034483,0.500000,0.064516,0.000000,0.0,0.000000,0.034483,0.500000,0.064516
